In [1]:
import pandas as pd
import numpy as np
import wrangle

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from scipy import stats
from math import sqrt

import sklearn.preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE

## Wrangle My Data:
- Acquire my dataframe
- Prepare/Clean my Dataframe
- Complete initial exploration to verify.

In [2]:
# acquire my dataframe from sql
df = wrangle.get_zillow()

In [3]:
# use my info function to find out more information
wrangle.get_info(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38396 entries, 0 to 38395
Data columns (total 8 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bedroomcnt                    38396 non-null  float64
 1   bathroomcnt                   38396 non-null  float64
 2   calculatedfinishedsquarefeet  38288 non-null  float64
 3   taxvaluedollarcnt             38395 non-null  float64
 4   yearbuilt                     38264 non-null  float64
 5   taxamount                     38395 non-null  float64
 6   fips                          38396 non-null  float64
 7   parcelid                      38396 non-null  int64  
dtypes: float64(7), int64(1)
memory usage: 2.3 MB
None
------------------------
------------------------
------------------------
This dataframe has 38396 rows and 8 columns.
------------------------\
Null count in dataframe:
------------------------
bedroomcnt                        0
bathroomcnt    

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,parcelid
11353,3.0,2.0,1837.0,505078.0,1976.0,6234.68,6037.0,11071338
36752,2.0,1.5,953.0,271756.0,1969.0,2742.76,6059.0,14621045
16286,4.0,3.0,2299.0,662740.0,1998.0,8058.16,6037.0,12644871


In [4]:
# let's rename our columns so they are more clear
df.rename(columns={'bedroomcnt': 'num_bedroom', 
                     'bathroomcnt': 'num_bathroom',
                     'calculatedfinishedsquarefeet': 'finished_sqft',
                     'taxvaluedollarcnt': 'tax_value',
                     'yearbuilt': 'build_year',
                     'taxamount': 'property_tax'}, inplace=True)

In [5]:
# since the nulls only account for .003 of the dataframe, let's drop nulls
df.dropna(inplace=True)
print(df.isnull().sum())
df.shape

num_bedroom      0
num_bathroom     0
finished_sqft    0
tax_value        0
build_year       0
property_tax     0
fips             0
parcelid         0
dtype: int64


(38258, 8)

In [6]:
df.drop_duplicates(inplace=True)
df.shape

(38226, 8)

In [7]:
# Tells us unique count
df.nunique()

num_bedroom         13
num_bathroom        19
finished_sqft     4187
tax_value        28238
build_year         131
property_tax     37317
fips                 3
parcelid         38226
dtype: int64

In [8]:
# now that we've been able to drop any houses with duplicate parcel ids, we can drop the column
df.drop(columns='parcelid', inplace=True)
df.describe().T

,count,mean,std,min,25%,50%,75%,max
num_bedroom,38226.0,3.022681,1.014806,0.00,2.00,3.00,4.0000,12.00
num_bathroom,38226.0,2.281745,0.943957,0.00,2.00,2.00,3.0000,11.00
finished_sqft,38226.0,1764.114582,927.452077,240.00,1174.00,1528.00,2085.0000,17245.00
tax_value,38226.0,496290.981348,629482.402761,1000.00,209401.50,364991.00,578708.5000,23858374.00
build_year,38226.0,1969.475749,23.102659,1878.00,1954.00,1970.00,1987.0000,2016.00
property_tax,38226.0,6041.119686,7319.556260,19.92,2728.97,4480.26,7001.7475,276797.83
fips,38226.0,6049.504630,20.922553,6037.00,6037.00,6037.00,6059.0000,6111.00


In [9]:
def value_counts(df, column):
    for col in column:
        print(col)
        print(df[col].value_counts())
        print('-------------')

In [16]:
column = df[df['num_bedroom', 'num_bathroom', 'fips', 'build_year']]
value_counts(df, column)

KeyError: ('num_bedroom', 'num_bathroom', 'fips', 'build_year')

In [17]:
df[df['num_bedroom', 'num_bathroom', 'fips', 'build_year']]

KeyError: ('num_bedroom', 'num_bathroom', 'fips', 'build_year')

In [14]:
df.head()

,num_bedroom,num_bathroom,finished_sqft,tax_value,build_year,property_tax,fips
0,3.0,2.0,1125.0,289445.0,1974.0,3490.16,6059.0
1,3.0,2.0,1316.0,205123.0,1923.0,2627.48,6037.0
2,3.0,2.0,1458.0,136104.0,1970.0,2319.90,6037.0
3,3.0,2.0,1766.0,810694.0,1980.0,9407.01,6037.0
4,2.0,1.0,1421.0,35606.0,1911.0,543.69,6037.0


In [ ]:
# To use when I want to concatenate my dfs back together
# df = pd.concat([x_train_scaled, y_train], axis=0)